## Setup

In [ ]:
import sys
sys.path.append("..")

In [ ]:
# Setup
import reproducibility

# both can change
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
# from keras.utils.vis_utils import plot_model
from keras.utils import plot_model

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import json
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K
# from keras.utils.vis_utils import plot_model
from scipy.interpolate import LinearNDInterpolator, interpn
from scipy.optimize import root
import pickle, os
from utils import hash2
%matplotlib inline

# Local modules for handling data and running moisture models
import data_funcs as datf
from data_funcs import format_raws, retrieve_raws, format_precip, fixnan, load_and_fix_data
from data_funcs import raws_data, synthetic_data, plot_data, check_data, rmse_data, to_json, from_json
import moisture_models as mod
from moisture_rnn import run_case, run_rnn, create_RNN_2, staircase, create_rnn_data, train_rnn, rnn_predict


# Testing Time Lag Parameter

The parameter $T_1 = 1/T$, where $T$ is the time lag fuel class (1h, 10h, 100h, 1000h). Changing this parameter should only affect how quickly the fmda model reacts to changes in the equilibrium moisture content. We test this with the following setup:

The equilibrium moisture content is simply a step function from 0 to 10 %. Assume this represents the true fuel moisture content.

In [ ]:
hours = 200
Ed = np.zeros(hours)
Ed[100:200] = 10
Ew = Ed
fm = Ew
rain = np.zeros(hours)

plt.plot(Ed)
plt.xlabel('Hour')
plt.ylabel('FM %')

In [ ]:
# Build Case Data
case_data = {
    'Ew': Ew,
    'Ed': Ed,
    'rain': rain,
    'fm': fm,
    'hours': hours,
    'h2': 100
}

We then setup an RNN with linear activation and 1 hidden unit, then for different fuel classes:
* Param set 1: 1h fuel
* Param set 10: 10h fuel
* Param set 100: 100h fuel

In [ ]:
param_sets={}
# Linear Activation Params
param_sets.update({1:{'id':1,
        'purpose':'test 1',
        'cases':'all',
        'batch_type':2,
        'training':5,
        'scale':1,        # every feature in [0, scale]
        'rain_do':True,
        'verbose':False,
        'timesteps':5,
        'activation':['linear','linear'],
        'hidden_units':1,  
        'dense_units':1,    # do not change
        'dense_layers':1,   # do not change
        'centering':[0.0,0.0],  # should be activation at 0
        'DeltaE':[0,-1],    # bias correction
        'synthetic':False,  # run also synthetic cases
        'T1': 1,          # 1/fuel class (1)
        'fm_raise_vs_rain': 0.17,         # fm increase per mm rain 
        'train_frac':0.5,  # time fraction to spend on training
        'epochs':1000,
        'verbose_fit':False,
        'verbose_weights':False,
}})
param_sets.update({10:{'id':10,
        'purpose':'test 1',
        'cases':'all',
        'batch_type':2,
        'training':5,
        'scale':1,        # every feature in [0, scale]
        'rain_do':True,
        'verbose':False,
        'timesteps':5,
        'activation':['linear','linear'],
        'hidden_units':1,  
        'dense_units':1,    # do not change
        'dense_layers':1,   # do not change
        'centering':[0.0,0.0],  # should be activation at 0
        'DeltaE':[0,-1],    # bias correction
        'synthetic':False,  # run also synthetic cases
        'T1': .1,          # 1/fuel class (10)
        'fm_raise_vs_rain': 0.17,         # fm increase per mm rain 
        'train_frac':0.5,  # time fraction to spend on training
        'epochs':1000,
        'verbose_fit':False,
        'verbose_weights':False,
}})
param_sets.update({100:{'id':100,
        'purpose':'test 1',
        'cases':'all',
        'batch_type':2,
        'training':5,
        'scale':1,        # every feature in [0, scale]
        'rain_do':True,
        'verbose':False,
        'timesteps':5,
        'activation':['linear','linear'],
        'hidden_units':1,  
        'dense_units':1,    # do not change
        'dense_layers':1,   # do not change
        'centering':[0.0,0.0],  # should be activation at 0
        'DeltaE':[0,-1],    # bias correction
        'synthetic':False,  # run also synthetic cases
        'T1': .01,          # 1/fuel class (100)
        'fm_raise_vs_rain': 0.17,         # fm increase per mm rain 
        'train_frac':0.5,  # time fraction to spend on training
        'epochs':1000,
        'verbose_fit':False,
        'verbose_weights':False,
}})

In [ ]:
print('~'*40)
print('Running 1h fuels')
rnn_dat = create_rnn_data(case_data,param_sets[1])
model1 = train_rnn(
        rnn_dat,
        param_sets[1],
        rnn_dat['hours'],
        fit=False
    )
case_data['m'] = rnn_predict(model1, param_sets[1], rnn_dat)
fit1 = case_data['m']
print('~'*40)
print('Running 10h fuels')
rnn_dat10 = create_rnn_data(case_data,param_sets[10])
model1 = train_rnn(
        rnn_dat10,
        param_sets[10],
        rnn_dat['hours'],
        fit=False
    )
case_data['m'] = rnn_predict(model1, param_sets[10], rnn_dat10)
fit10 = case_data['m']
print('~'*40)
print('Running 100h fuels')
rnn_dat100 = create_rnn_data(case_data,param_sets[100])
model1 = train_rnn(
        rnn_dat100,
        param_sets[100],
        rnn_dat['hours'],
        fit=False
    )
case_data['m'] = rnn_predict(model1, param_sets[100], rnn_dat100)
fit100 = case_data['m']

In [ ]:
plt.plot(fm)
plt.plot(fit1, label="1-h")
plt.plot(fit10, label="10-h")
plt.plot(fit100, label="100-h")
plt.legend()

In [ ]:
print(rnn_dat['scale_fm'])
print(rnn_dat10['scale_fm'])
print(rnn_dat100['scale_fm'])